In [0]:
%sql
USE CATALOG initialcatalog;
USE SCHEMA initialschema;

SHOW TABLES;

In [0]:
spark.catalog.setCurrentCatalog("initialcatalog")
spark.catalog.setCurrentDatabase("initialschema")
spark.catalog.listTables("initialschema")
spark.catalog.listTables()

In [0]:
spark.sql(f"LIST '/Volumes/initialcatalog/initialschema/initialvolume'").display()

In [0]:
%sql
USE CATALOG initialcatalog;
-- CTAS -- Create table as
DROP TABLE IF EXISTS initialcatalog.initialschema.electric_vehicles;
create TABLE initialschema.electric_vehicles as
SELECT `VIN (1-10)` AS VIN ,
County,
City,
State,
`Postal Code` AS Postal_Code,
`Model Year` AS Model_Year,
Make,
Model,
`Electric Vehicle Type` AS Vehicle_Type,
`Clean Alternative Fuel Vehicle (CAFV) Eligibility` AS CAFV_Eligibility,
`Electric Range` AS Range,
`Base MSRP` AS Base_MSRP,
`Legislative District` AS Legislative_District,
`DOL Vehicle ID` AS DOL_Vehicle_ID,
`Vehicle Location` AS Vehicle_Location,
`Electric Utility` as Electric_Utility,
`2020 Census Tract` AS 2020_Census_Tract 
FROM 
read_files(
  '/Volumes/initialcatalog/initialschema/initialvolume/Electric_Vehicle_Population_Data.csv',
  format => 'csv',
 sep => ',',
  inferschema => True,
  header => True
);

In [0]:
%sql
USE CATALOG initialcatalog;
USE SCHEMA initialschema;
-- CTAS -- Create table as
DROP TABLE IF EXISTS electric_vehicles_1;
CREATE TABLE electric_vehicles_1 (
    VIN STRING,
    County STRING,
    City STRING,
    State STRING,
    Postal_Code STRING,
    Model_Year INT,
    Make STRING,
    Model STRING,
    Vehicle_Type STRING,
    CAFV_Eligibility STRING,
    Range INT,
    Base_MSRP DOUBLE,
    Legislative_District INT,
    DOL_Vehicle_ID INT,
    Vehicle_Location STRING,
    Electric_Utility STRING,
    2020_Census_Tract LONG
) ;

copy into electric_vehicles_1
from (
  SELECT `VIN (1-10)` AS VIN ,
County,
City,
State,
CAST(`Postal Code` as STRING) AS Postal_Code,
`Model Year` AS Model_Year,
Make,
Model,
`Electric Vehicle Type` AS Vehicle_Type,
`Clean Alternative Fuel Vehicle (CAFV) Eligibility` AS CAFV_Eligibility,
`Electric Range` AS Range,
CAST(`Base MSRP` AS double) AS Base_MSRP,
`Legislative District` AS Legislative_District,
`DOL Vehicle ID` AS DOL_Vehicle_ID,
`Vehicle Location` AS Vehicle_Location,
`Electric Utility` as Electric_Utility,
CAST(`2020 Census Tract` AS LONG) AS 2020_Census_Tract 
  from
  '/Volumes/initialcatalog/initialschema/initialvolume/Electric_Vehicle_Population_Data.csv')
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true', 'mergeSchema' = 'true');
    

In [0]:
%sql
select * from electric_vehicles_1 limit 10;

In [0]:
%sql
--MERGE Script
USE CATALOG initialcatalog;
USE SCHEMA initialschema;

list "/Volumes/initialcatalog/initialschema/initialvolume";

CREATE TABLE IF NOT EXISTS students(
ID INT,
Name STRING
);

copy into students from
(
  select cast(ID as INT), CAST(NAME as STRING)
  from '/Volumes/initialcatalog/initialschema/initialvolume/Student.csv'
)
FILEFORMAT=CSV
FORMAT_OPTIONS(
  'header' = 'true',
  'mergeSchema' = 'true',
  'separator' = '|',
  'encoding' = 'UTF-8',
  'inferSchema' = 'true'
)


In [0]:
%sql
 USE CATALOG initialcatalog;
USE SCHEMA initialschema;
 CREATE TABLE IF NOT EXISTS students(
ID INT,
Name STRING
);
 merge into students as  t
 using (
   
 select cast(ID as INT), CAST(NAME as STRING)
  from read_files('/Volumes/initialcatalog/initialschema/initialvolume/Student.csv',
  format => 'csv',
  sep => '|',
  header => True,
  inferSchema => True
  )
 ) as s
 on t.ID = s.ID
 when matched then update set 
  t.NAME = s.NAME
 when not matched then insert 
  (ID, Name)
  values (s.ID, s.NAME)
  WHEN NOT MATCHED BY SOURCE THEN DELETE;

In [0]:
%sql
select * from students;

In [0]:
%sql
describe history students;
select* from students version as of 5;